In [2]:
from torchvision import transforms

# Define your transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
import torch
from torch import nn
from torchvision import datasets, transforms

# Load the pre-trained model
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

# Adjust the final layer to match the number of classes in your dataset
train_dataset = datasets.ImageFolder(root='dataset/train', transform=transform)
num_classes = len(train_dataset.classes)
model.head = nn.Linear(model.head.in_features, num_classes)

# Load your custom model weights
model_path = 'weight/deit_fungus25_epoch100.pt'
model.load_state_dict(torch.load(model_path))

# Prepare sample input for model
batch_size = 1
channels = 3
height = 224  # Make sure this matches the expected input size
width = 224
sample_input = torch.rand((batch_size, channels, height, width))

# Path for the ONNX model
output_onnx = "weight/deit_fungus25_epoch100.onnx"  # Corrected the file extension to .onnx

torch.onnx.export(model, sample_input, output_onnx, export_params=True)

Using cache found in /home/samxu/.cache/torch/hub/facebookresearch_deit_main
/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/torch/__init__.py:1404: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


In [4]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model_onnx = onnx.load(output_onnx)

# Prepare the TensorFlow representation
tf_rep = prepare(model_onnx)

# Export the TensorFlow model
tf_rep.export_graph('model_tf')


2023-11-24 02:25:59.006563: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 02:25:59.036325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 02:25:59.553616: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance

KeyError: in user code:

    File "/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/onnx_tf/backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/onnx_tf/backend.py", line 347, in _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    File "/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/onnx_tf/handlers/handler.py", line 59, in handle  *
        return ver_handle(node, **kwargs)
    File "/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/onnx_tf/handlers/backend/conv.py", line 15, in version_11  *
        return cls.conv(node, kwargs["tensor_dict"])
    File "/home/samxu/projects/apply_ai/final_fungus-detector/.venv/lib/python3.8/site-packages/onnx_tf/handlers/b